In [1]:
import pandas as pd #data analysis
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("Retail_Data_Transactions.csv") #import

#preview
df.head()

,customer_id,trans_date,tran_amount
0,CS5295,11-Feb-13,35
1,CS4768,15-Mar-15,39
2,CS2122,26-Feb-13,52
3,CS1217,16-Nov-11,99
4,CS1850,20-Nov-13,78


In [3]:
#datatypes
df.dtypes

customer_id    object
trans_date     object
tran_amount     int64
dtype: object

In [4]:
#change date to date
df.trans_date = pd.to_datetime(df["trans_date"])

#data type
df.dtypes

customer_id            object
trans_date     datetime64[ns]
tran_amount             int64
dtype: object

In [5]:
#preview
df.head(2)

,customer_id,trans_date,tran_amount
0,CS5295,2013-02-11,35
1,CS4768,2015-03-15,39


### RFM
    - Recency

In [6]:
#Aggregate for the min and max date for each customer IDD
recency = df.groupby("customer_id").agg({'trans_date':['min', 'max']}).reset_index()

#drop levels on columns
recency.columns = recency.columns.droplevel()
    
#rename columns
recency.columns = ["ID", "Min", "Max"]

#preview
recency.head()

,ID,Min,Max
0,CS1112,2011-06-15,2015-01-14
1,CS1113,2011-05-27,2015-02-09
2,CS1114,2011-07-14,2015-02-12
3,CS1115,2011-08-10,2015-03-05
4,CS1116,2011-06-27,2014-08-25


In [7]:
df.dtypes

customer_id            object
trans_date     datetime64[ns]
tran_amount             int64
dtype: object

In [8]:
import datetime as dt
PRESENT = dt.datetime(2011,12,10)

In [9]:
df.groupby("customer_id").agg({"trans_date": lambda date: (date.max() - date.min()).days}).reset_index()

,customer_id,trans_date
0,CS1112,1309
1,CS1113,1354
2,CS1114,1309
3,CS1115,1303
4,CS1116,1155
...,...,...
6884,CS8996,1120
6885,CS8997,1116
6886,CS8998,1293
6887,CS8999,1073


In [10]:
type(df)

pandas.core.frame.DataFrame

In [18]:
df.trans_date.max()

Timestamp('2015-03-16 00:00:00')

In [20]:
present = dt.datetime(2015,3,17)

present

datetime.datetime(2015, 3, 17, 0, 0)

- Investigate how recency max date is actually calculated.  **
- See how classification is done
- Set list for segmentation for user customizability, also default to 4 segments with warning message.

- Note that for Recency Score we need to give inverse labels as the more active the customer is the lower the value of Recency. :)

- Customers with the lowest recency, highest frequency and monetary amounts considered as top customers.

In [115]:
def recency(data, id, date, present_date, bins = {5 : [5,4,3,2,1]}):
    '''
    This function will calculate the recency.
    data: Dataframe
        
        A dataframe containing at least the customer ID, Transaction/Order Date
    
    id:   String, Integer
        
        (Order/Customer/Transaction) Id column (string or int)
    
    date: Date, Datetime, Object

        A date column (NB: A type conversion will be attempted by the function but you can also ensure the date type is proper)
    threshold: An integer value, specifying how many days should be considered
    
    present_date: datetime 
        The present date or most recent date to serve as a reference point for recency
        
    bins: dictionary, default 5
        A dictionary containing the number of segments to be created from recency score and the labels associated 
        to quantile groups
    '''
    #error instance to check if input is either a pandas dataframe and is also not none
    if isinstance(data, pd.DataFrame) == False or data is None:
        raise ValueError("data: Expecting a Dataframe or got 'None'")
    
    #error instance to check if id column is an identified column name
    if id not in data.columns:
        raise ValueError("id: Expected an id (a column) in Dataframe")
    
    #examine the datatype of the date column
    if data.dtypes[date].name != "datetime64[ns]":
        raise ValueError("date: Expected a date datatype, 'convert the date/datetime type'")
        #print("This Column is not a of date/datetime data type")
    
    if present_date is None:
        raise ValueError("present_date: Specify the recent date to calculate recency")
    #aggregation to calculate recency
    result = data.groupby(id) \
            .agg({date: lambda date: (present_date - date.max()).days})\
            .reset_index() #reset index
    
    if isinstance(bin, int) == False:
        raise ValueError("bin: Value needs to be an integer. default is 5")
    
    if len(list(bins.values())) != int(list(bins.keys())[0]):
        print("Warning: The number of bins for quantile is not same as the label. default is {}".format(bins))
    
    #rename column
    result.rename(columns = {date : "recency"}, inplace = True)
    
    #get bin value
    bin_value = int(list(bins.keys())[0])
    
    #get bin labels
    bin_labels = list(bins.values())[0]
    
    #logic to use bins value
    result["recency_bins"] = pd.qcut(result.recency, bin_value, bin_labels)
    
    #note that if bin changes then label needs to change also
    
    
    return result

In [120]:
bins = {4: [4,3,2,1]}
#get bin value
bin_value = int(list(bins.keys())[0])
    
#get bin labels
bin_labels = list(bins.values())



In [122]:
rec = recency(df, "customer_id", "trans_date", present, bins = {6: [6,5,4,3,2,1]})

rec.recency_bins.value_counts()

5    1175
6    1164
2    1151
3    1149
1    1143
4    1107
Name: recency_bins, dtype: int64

In [31]:
pd.qcut(rec.recency, 5, [5,4,3,2,1]).value_counts()

5    1445
3    1374
1    1374
2    1363
4    1333
Name: recency, dtype: int64

In [24]:
type(present)

datetime.datetime

#### Frequency

In [13]:
def frequency(data, id):
    '''
    This function will calculate the recency.
    data: Dataframe
        
        A dataframe containing at least the customer ID, Transaction/Order Date
    
    id:   String, Integer
        
        (Order/Customer/Transaction) Id column (string or int)
    '''
    #error instance to check if input is either a pandas dataframe and is also not none
    if isinstance(data, pd.DataFrame) == False or data is None:
        raise ValueError("data: Expecting a Dataframe or got 'None'")
    
    #error instance to check if id column is an identified column name
    if id not in data.columns:
        raise ValueError("id: Expected an id (a column) in Dataframe")
    
    #examine the datatype of the date column
    #aggregation to calculate recency
    result = pd.DataFrame(data.groupby(id).size()).reset_index()
    
    
    #rename column
    result.rename(columns = {0 : "frequency"}, inplace = True)
    return result

In [14]:
def monetary(data, id, spend):
    '''
    This function will calculate the recency.
    data: Dataframe
        
        A dataframe containing at least the customer ID, Transaction/Order Date
    
    id:   String, Integer
        
        (Order/Customer/Transaction) Id column (string or int)
        
    spend: int or float
        
        The cost column either transaction cost
    '''
    #error instance to check if input is either a pandas dataframe and is also not none
    if isinstance(data, pd.DataFrame) == False or data is None:
        raise ValueError("data: Expecting a Dataframe or got 'None'")
    
    #error instance to check if id column is an identified column name
    if id not in data.columns:
        raise ValueError("id: Expected an id (a column) in Dataframe")
    
    #examine the datatype of the date column
    #aggregation to calculate recency
    result = pd.DataFrame(data.groupby(id)[spend].sum()).reset_index()
    
    
    #rename column
    result.rename(columns = {spend : "monetary"}, inplace = True)
    return result

In [15]:
monetary(df, "customer_id", "tran_amount")

,customer_id,monetary
0,CS1112,1012
1,CS1113,1490
2,CS1114,1432
3,CS1115,1659
4,CS1116,857
...,...,...
6884,CS8996,582
6885,CS8997,543
6886,CS8998,624
6887,CS8999,383


In [16]:
pd.DataFrame(df.groupby("customer_id")["tran_amount"].sum()).reset_index()

,customer_id,tran_amount
0,CS1112,1012
1,CS1113,1490
2,CS1114,1432
3,CS1115,1659
4,CS1116,857
...,...,...
6884,CS8996,582
6885,CS8997,543
6886,CS8998,624
6887,CS8999,383


In [17]:
test = pd.read_csv("https://raw.githubusercontent.com/simmieyungie/Bakery-Analytics/master/transactionsAHW.csv")

test.head()

,OnlineOrderFlag,CustomerID,Name,ProductSubName,ProductCatName,OrderQty,OrderDate
0,True,11000,Fender Set - Mountain,Fenders,Accessories,1,2013-06-20
1,True,11000,"Mountain-100 Silver, 38",Mountain Bikes,Bikes,1,2011-06-21
2,True,11000,Touring Tire Tube,Tires and Tubes,Accessories,1,2013-10-03
3,True,11000,"Mountain-200 Silver, 38",Mountain Bikes,Bikes,1,2013-06-20
4,True,11000,"Sport-100 Helmet, Red",Helmets,Accessories,1,2013-10-03
